## Spark Stream Practice

In [1]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0bc4bd12-1b8f-42c3-8e1b-9edb4ee836a0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 173ms :: artifacts dl 4m

### Spark Stream with socket

In another tab open a terminal and open a socket using 
    
    nc -l -p 9090

This will allow you to open a TCP socket and to manually send data

Connect Spark to the stream

In [3]:
lines = (spark.readStream                        
.format("socket")                                 
.option("host","localhost")
.option("port","9090")
.load())   

25/10/24 12:58:00 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


For now, let's just inspect the stream

In [2]:
lines.writeStream \
          .outputMode("append") \
          .format("console") \
          .start()



NameError: name 'lines' is not defined

Now let's do a word count

In [4]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

 # Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

25/10/24 12:58:14 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3c8fc314-35b8-4b7f-a844-9ba8098821b5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    3|
|world|    2|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    3|
|world|    3|
+-----+-----+



KeyboardInterrupt: 

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

 # Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("update") \
          .format("console") \
          .start()

query.awaitTermination()

In [ ]:
# Change the outputMode to "update" and notice how it change

### Spark streaming with a Kafka

We will use the other notebook "Kafka-producer" to handle Kafka

In [2]:
kafka_server = "kafka1:9092"   

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "words")                       # Subscribe to the "en" Kafka topic - edits of English wikipedia pages
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()                                          # Load the DataFrame
)

In [3]:
# Task: Use the code written for the previous case to do count the words in the stream
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

 # Generate running word count
wordCounts = words.groupBy("word").count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

25/10/24 13:00:50 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9d730ef7-bbfb-462f-a1b5-a9bcf76e8ae5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    1|
|         Sit|    1|
|     tempora|    7|
|consectetur.|    1|
|         Sed|    1|
|        non.|    2|
|       velit|    5|
|   adipisci.|    3|
|         non|    5|
|     Aliquam|    2|
|     dolore.|    2|
|         sit|    7|
|       Porro|    4|
| consectetur|   11|
|      Labore|    1|
|    quisquam|    5|
|    tempora.|    4|
|    numquam.|    2|
|    Adipisci|    1|
|      Dolore|    1|
+------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 1
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    1|
|         Sit|    1|
|     tempora|    7|
|consectetur.|    1|
|         Sed|    1|
|        non.|    2|
|       velit|    5|
|   adipisci.|    4|
|         non|    5|
|     Aliquam|    2|
|     dolore.|    2|
|         sit|    8|
|       Porro|    4|
| consectetur|   12|
|      Labore|    2|
|    quisquam|    6|
|    numquam.|    2|
|    tempora.|    6|
|    Adipisci|    2|
|      Dolore|    1|
+------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 2
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    1|
|         Sit|    2|
|     tempora|    7|
|consectetur.|    2|
|         Sed|    1|
|        non.|    2|
|       velit|    5|
|   adipisci.|    4|
|         non|    5|
|     Aliquam|    2|
|     dolore.|    2|
|         sit|    8|
|       Porro|    4|
| consectetur|   12|
|      Labore|    3|
|    quisquam|    8|
|    numquam.|    2|
|    tempora.|    6|
|    Adipisci|    2|
|      Dolore|    1|
+------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 3
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    1|
|         Sit|    2|
|     tempora|    7|
|consectetur.|    2|
|         Sed|    1|
|        non.|    2|
|       velit|    5|
|   adipisci.|    4|
|         non|    6|
|     Aliquam|    2|
|     dolore.|    2|
|         sit|    8|
|       Porro|    4|
| consectetur|   12|
|      Labore|    4|
|    quisquam|    9|
|    numquam.|    2|
|    tempora.|    6|
|    Adipisci|    2|
|      Dolore|    1|
+------------+-----+
only showing top 20 rows



...

-------------------------------------------
Batch: 30
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    4|
|         Sit|    4|
|     tempora|   13|
|consectetur.|    5|
|         Sed|    2|
|        non.|    2|
|      ipsum.|    2|
|       velit|   13|
|   adipisci.|    4|
|         non|   13|
|     Aliquam|    3|
|     dolore.|    7|
|         sit|   15|
|       Porro|    6|
| consectetur|   23|
|      Labore|    5|
|    quisquam|   21|
|    numquam.|    4|
|    tempora.|   12|
|    Adipisci|    5|
+------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 31
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    4|
|         Sit|    4|
|     tempora|   14|
|consectetur.|    5|
|         Sed|    2|
|        non.|    2|
|      ipsum.|    2|
|       velit|   13|
|   adipisci.|    4|
|         non|   13|
|     Aliquam|    3|
|     dolore.|    8|
|         sit|   15|
|       Porro|    6|
| consectetur|   24|
|      Labore|    5|
|    quisquam|   21|
|    numquam.|    4|
|    tempora.|   12|
|    Adipisci|    6|
+------------+-----+
only showing top 20 rows



KeyboardInterrupt: 

-------------------------------------------
Batch: 32
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    4|
|         Sit|    5|
|     tempora|   15|
|consectetur.|    6|
|         Sed|    2|
|        non.|    2|
|      ipsum.|    2|
|       velit|   13|
|   adipisci.|    4|
|         non|   13|
|     Aliquam|    3|
|     dolore.|    8|
|         sit|   17|
|       Porro|    6|
| consectetur|   24|
|      Labore|    5|
|    quisquam|   22|
|    numquam.|    4|
|    tempora.|   12|
|    Adipisci|    6|
+------------+-----+
only showing top 20 rows



-------------------------------------------
Batch: 33
-------------------------------------------
+------------+-----+
|        word|count|
+------------+-----+
|    Quisquam|    4|
|         Sit|    5|
|     tempora|   15|
|consectetur.|    6|
|         Sed|    2|
|        non.|    2|
|      ipsum.|    2|
|       velit|   13|
|   adipisci.|    4|
|         non|   13|
|     Aliquam|    3|
|     dolore.|    8|
|         sit|   18|
|       Porro|    6|
| consectetur|   25|
|      Labore|    5|
|    quisquam|   23|
|    numquam.|    4|
|    tempora.|   13|
|    Adipisci|    6|
+------------+-----+
only showing top 20 rows



### Spark Structured Stream

Let's start with an easy example that reads from file

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType

schema = StructType(
      [
        StructField("RecordNumber", IntegerType(), True),
        StructField("Zipcode", StringType(), True),
        StructField("ZipCodeType", StringType(), True),
        StructField("City", StringType(), True),
        StructField("State", StringType(), True),
        StructField("LocationType", StringType(), True),
        StructField("Lat", StringType(), True),
        StructField("Long", StringType(), True),
        StructField("Xaxis", StringType(), True),
        StructField("Yaxis", StringType(), True),
        StructField("Zaxis", StringType(), True),
        StructField("WorldRegion", StringType(), True),
        StructField("Country", StringType(), True),
        StructField("LocationText", StringType(), True),
        StructField("Location", StringType(), True),
        StructField("Decommisioned", StringType(), True)
      ]
    )

In [ ]:
zips = spark.readStream.schema(schema).json('data/')

In [ ]:
zips.printSchema()

We can look at the stream

In [ ]:
zips.writeStream \
          .outputMode("append") \
          .format("console") \
          .start()

And we can compute some aggregation on it

In [ ]:
groupDF = zips.select("Zipcode").groupBy("Zipcode").count()

#groupDF.printSchema()

groupDF.writeStream \
.format("console") \
.outputMode("complete") \
.start() \
.awaitTermination()

In [ ]:
# Change the code above to aggregate the location according to different categories (e.g., location type) 
groupDF = zips.select("City").groupBy("City").count()

#groupDF.printSchema()

groupDF.writeStream \
.format("console") \
.outputMode("complete") \
.start() \
.awaitTermination()

### Window operator

In [ ]:
from pyspark.sql.functions import window

kafka_server = "kafka1:9092"   

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "words")                       # Subscribe to the "words" Kafka topic
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()                                          # Load the DataFrame
)

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds")).count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

#### Sliding time window

In [ ]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds", "2 seconds")).count()

 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

#### Other aggregations

In [ ]:
# Knowing that you can use the method "orderBy(col("columname").desc())" to order the element in the batch
# modiffy the code above to rank the words in order of appearance

# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"),col("timestamp"))

 # Generate running word count
wordCounts = words.groupBy("word", window("timestamp", "5 seconds", "2 seconds")).count()

orderedWords = wordCounts.orderBy(col("count").desc())

 # Start running the query that prints the running counts to the console
query = orderedWords \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()


### Wikipedia edits

https://stream.wikimedia.org/v2/ui/#/?streams=mediawiki.recentchange

Spend some minutes to look how are the data on the website and look at the schema below (we won't consider all the fields)

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType

schema = StructType(
      [
        StructField("domain", StringType(), True),
        StructField("namespace", StringType(), True),
        StructField("uri", StringType(), True),
        StructField("title", StringType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("user_name", StringType(), True),
        StructField("user_type", StringType(), True),
        StructField("old_length", StringType(), True),
        StructField("new_length", StringType(), True),
      ]
    )

In [4]:
from pyspark.sql.functions import window
from pyspark.sql.functions import from_json
kafka_server = "kafka1:9092"   

wiki_df = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "edit")                       # Subscribe to the "en" Kafka topic - edits of English wikipedia pages
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
)

In the last mimute how many edits are done by bots and by humans

In [5]:
data = wiki_df.select("parsed_value.*")

group_df = data.groupBy(col("user_type"), window(col("timestamp"),"1 minutes")).count()

query = group_df \
          .writeStream \
          .outputMode("update") \
          .format("console") \
          .start()

query.awaitTermination()

25/10/24 14:00:55 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-df7985a7-b5c4-4935-9b5c-2558366a3bfb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 13:59...|   15|
|    human|[2025-10-24 13:59...|   85|
+---------+--------------------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 13:59...|   34|
|    human|[2025-10-24 13:59...|  166|
+---------+--------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|    human|[2025-10-24 13:59...|  167|
|      bot|[2025-10-24 14:00...|   14|
|    human|[2025-10-24 14:00...|   85|
+---------+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 14:00...|   34|
|    human|[2025-10-24 14:00...|  165|
+---------+--------------------+-----+



...

-------------------------------------------
Batch: 25
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 14:01...|   93|
|    human|[2025-10-24 14:01...|  313|
+---------+--------------------+-----+



-------------------------------------------
Batch: 26
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 14:01...|   98|
|    human|[2025-10-24 14:01...|  334|
+---------+--------------------+-----+



-------------------------------------------
Batch: 27
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 14:01...|  104|
|    human|[2025-10-24 14:01...|  345|
+---------+--------------------+-----+



-------------------------------------------
Batch: 28
-------------------------------------------
+---------+--------------------+-----+
|user_type|              window|count|
+---------+--------------------+-----+
|      bot|[2025-10-24 14:01...|  107|
|    human|[2025-10-24 14:01...|  353|
+---------+--------------------+-----+



KeyboardInterrupt: 

Select the 10 most edited pages

In [ ]:
from pyspark.sql.functions import col, row_number

data = wiki_df.select("parsed_value.*")

group_df = data.groupBy(col("uri"), window(col("timestamp"),"1 minute")).count()

ordered_df = group_df.orderBy(col("count").desc()).limit(10)

query = ordered_df \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()


Who are the top editors of the english wikipedia?

In [ ]:
from pyspark.sql.functions import col, row_number

data = wiki_df.select("parsed_value.*")

group_df = data.filter(col("domain")=="en.wikipedia.org").groupBy(col("user_name"), window(col("timestamp"),"1 minute")).count()

ordered_df = group_df.orderBy(col("count").desc()).limit(10)

query = ordered_df \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()

Are they humans or bot?

In [ ]:
from pyspark.sql.functions import col, row_number

data = wiki_df.select("parsed_value.*")

group_df = data.filter(col("domain")=="en.wikipedia.org").groupBy(col("user_name"),col("user_type"), window(col("timestamp"),"1 minute")).count()

ordered_df = group_df.orderBy(col("count").desc()).limit(10)

query = ordered_df \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .start()

query.awaitTermination()



How many edits added something to the page?

In [ ]:
# Hint: withColumn() allows you to create a new column in the dataframe
# Hint: when() and otherwise() allow you to insert value in column when some condition are satisfied
# e.g, when((condition_1), value).otherwise(value_2)
from pyspark.sql.functions import col, row_number, when, lit

data = wiki_df.select("parsed_value.*")

group_df = data \
.withColumn("op_type", when((data.new_length - data.old_length > 0),lit("ADD")).otherwise(lit("REMOVE"))) \
.groupBy(col("op_type")).count()

query = group_df \
          .writeStream \
          .outputMode("update") \
          .format("console") \
          .start()

query.awaitTermination()


#### Joining streams

Let's register to the "new" stream

In [4]:
from pyspark.sql.functions import window
from pyspark.sql.functions import from_json
kafka_server = "kafka1:9092"   

new_df = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "new")                       # Subscribe to the "en" Kafka topi
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
)

And let's the if there are users who add and then modify

In [5]:
df_edit = wiki_df.select("parsed_value.*").select("user_name","user_type","uri","title","timestamp")

df_new = new_df.select("parsed_value.*").select("user_name","user_type","uri","title","timestamp")

joined_streams_df = (df_edit
            .join(df_new, "user_name")
            )

query = joined_streams_df.writeStream.outputMode("append").format("console").start()

query.awaitTermination()

24/11/12 15:24:19 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-70cf9ec9-0204-4bce-8177-4213c90f940c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+---------+---+-----+---------+---------+---+-----+---------+
|user_name|user_type|uri|title|timestamp|user_type|uri|title|timestamp|
+---------+---------+---+-----+---------+---------+---+-----+---------+
+---------+---------+---+-----+---------+---------+---+-----+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------+-------------------+
|   user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|         title|          timestamp|
+------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------+-------------------+
| Warburg1866|    human|https://de.wikipe...|     Andreas Magdanz|2024-11-07 16:33:08|    human|https://commons.w...|Category:Ogbia|2024-11-07 16:51:06|
|     Arjoopy|    human|https://commons.w...|File:Meiningen 19...|2024-11-07 16:33:18|    human|https://www.wikid...|    Q131013792|2024-11-07 16:50:16|
|Dorcas Atule|    human|https://www.wikid...|          Q131013068|2024-11-07 16:33:16|    human|https://www.wikid...|    Q131013723|2024-1

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------+---------+--------------------+-----------+-------------------+---------+--------------------+--------------------+-------------------+
|         user_name|user_type|                 uri|      title|          timestamp|user_type|                 uri|               title|          timestamp|
+------------------+---------+--------------------+-----------+-------------------+---------+--------------------+--------------------+-------------------+
|InternetArchiveBot|      bot|https://vi.wikipe...|Ga Cheolsan|2024-11-07 16:33:17|      bot|https://ckb.wikip...|وتووێژ:محەممەدڕەز...|2024-11-07 16:52:12|
|      Dorcas Atule|    human|https://www.wikid...| Q131013068|2024-11-07 16:33:16|    human|https://www.wikid...|          Q131013858|2024-11-07 16:52:07|
|          Sławobóg|    human|https://www.wikid...|  Q66962308|2024-11-07 16:33:25|    human|https://hsb.wikti...|Modul:mes

-------------------------------------------
Batch: 3
-------------------------------------------
+----------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
| user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|               title|          timestamp|
+----------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|  Hameryko|    human|https://www.wikid...|     Lexeme:L1376306|2024-11-07 16:33:29|    human|https://www.wikid...|     Lexeme:L1376321|2024-11-07 16:48:39|
|  Hameryko|    human|https://www.wikid...|     Lexeme:L1376306|2024-11-07 16:33:29|    human|https://www.wikid...|     Lexeme:L1376322|2024-11-07 16:49:00|
|  Hameryko|    human|https://www.wikid...|     Lexeme:L1376306|2024-11-07 16:33:29|    human|https://www.wikid...|   

...

-------------------------------------------
Batch: 141
-------------------------------------------
+------------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|         user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|               title|          timestamp|
+------------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|              Qcne|    human|https://en.wikipe...|Draft:The Childre...|2024-11-12 14:29:30|    human|https://en.wikipe...|   User talk:Zuraged|2024-11-07 16:48:47|
|InternetArchiveBot|      bot|https://ru.wikipe...|               Хэбэй|2024-11-12 14:29:28|      bot|https://ckb.wikip...|وتووێژ:محەممەدڕەز...|2024-11-07 16:52:12|
|         Hagougaga|    human|https://www.wikid...|          Q131153668|2024

-------------------------------------------
Batch: 142
-------------------------------------------
+------------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|         user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|               title|          timestamp|
+------------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|InternetArchiveBot|      bot|https://uk.wikipe...|Срібний м'яч (Бра...|2024-11-12 14:29:26|      bot|https://ckb.wikip...|وتووێژ:محەممەدڕەز...|2024-11-07 16:52:12|
|         Hagougaga|    human|https://www.wikid...|          Q131153671|2024-11-12 14:29:36|    human|https://www.wikid...|          Q131154035|2024-11-12 15:23:00|
|         Hagougaga|    human|https://www.wikid...|          Q131153671|2024

-------------------------------------------
Batch: 143
-------------------------------------------
+---------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|      user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|               title|          timestamp|
+---------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|    கி.மூர்த்தி|    human|https://ta.wikipe...|        அலையடிக்குது|2024-11-12 14:29:41|    human|https://ta.wikipe...|          சங்கரோத்து|2024-11-07 16:52:37|
|      Hagougaga|    human|https://www.wikid...|          Q131153673|2024-11-12 14:29:43|    human|https://www.wikid...|          Q131154035|2024-11-12 15:23:00|
|      Hagougaga|    human|https://www.wikid...|          Q131153673|2024-11-12 14:29:43|  

-------------------------------------------
Batch: 144
-------------------------------------------
+------------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|         user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|               title|          timestamp|
+------------------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|InternetArchiveBot|      bot|https://uk.wikipe...|       Срібний скарб|2024-11-12 14:29:44|      bot|https://ckb.wikip...|وتووێژ:محەممەدڕەز...|2024-11-07 16:52:12|
|         Hagougaga|    human|https://www.wikid...|          Q131153674|2024-11-12 14:29:51|    human|https://www.wikid...|          Q131154035|2024-11-12 15:23:00|
|         Hagougaga|    human|https://www.wikid...|          Q131153674|2024

-------------------------------------------
Batch: 145
-------------------------------------------
+---------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|user_name|user_type|                 uri|               title|          timestamp|user_type|                 uri|               title|          timestamp|
+---------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+
|Pierre cb|    human|https://commons.w...|File:Aral Sea 202...|2024-11-12 14:29:54|    human|https://fr.wikipe...|Gyre d'Amérique c...|2024-11-07 16:49:21|
+---------+---------+--------------------+--------------------+-------------------+---------+--------------------+--------------------+-------------------+

